# Models

Looking at the lower level API of Transformers - the models that wrap PyTorch code for the transformers themselves.

This notebook can run on a low-cost or free T4 runtime.

In [2]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

# Uninstall flash-attention (the PyPI package name may vary)
!pip uninstall flash-attn -y
!pip uninstall xformers -y

!pip install flash-attn
!pip install xformers


# Clean up the notebook
clean_notebook()



Notebook cleaned.


In [3]:

from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

# Sign in to Hugging Face

1. If you haven't already done so, create a free HuggingFace account at https://huggingface.co and navigate to Settings, then Create a new API token, giving yourself write permissions

2. Press the "key" icon on the side panel to the left, and add a new secret:
`HF_TOKEN = your_token`

3. Execute the cell below to log in.

In [4]:
import os
from huggingface_hub import login

hf_token = os.environ.get('HF_TOKEN')

# Log into Hugging Face, adding credentials to git if needed
login(hf_token, add_to_git_credential=False)

In [9]:
# instruct models

LLAMA    = "meta-llama/Meta-Llama-3.1-8B-Instruct"
Deepseek ="deepseek-ai/DeepSeek-R1-Distill-Llama-8B"


In [10]:
messages = [
    {"role": "system", "content": "You are a helpful assistant answer in only thai language"},
    {"role": "user", "content": "เล่านิทานให้ที in thai  เกี่ยวกับ แมว และ สุนัข และ เต่า"}
  ]

# Accessing Llama 3.1 from Meta

In order to use the fantastic Llama 3.1, Meta does require you to sign their terms of service.

Visit their model instructions page in Hugging Face:
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B

At the top of the page are instructions on how to agree to their terms. If possible, you should use the same email as your huggingface account.

In my experience approval comes in a couple of minutes. Once you've been approved for any 3.1 model, it applies to the whole family of models.

In [11]:
# Quantization Config - this allows us to load the model into memory and use less memory

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

If the next cell gives you a 403 permissions error, then please check:
1. Are you logged in to HuggingFace? Try running `login()` to check your key works
2. Did you set up your API key with full read and write permissions?
3. If you visit the Llama3.1 page at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B, does it show that you have access to the model near the top?


In [12]:
# Tokenizer

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [14]:
# The model
model = AutoModelForCausalLM.from_pretrained(
    LLAMA, 
    device_map="auto", 
    quantization_config=quant_config,
    
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f} MB")

Memory footprint: 5,591.5 MB


In [16]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [17]:
outputs = model.generate(inputs, max_new_tokens=500,eos_token_id = tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant answer in only thai language<|eot_id|><|start_header_id|>user<|end_header_id|>

เล่านิทานให้ที in thai  เกี่ยวกับ แมว และ สุนัข และ เต่า<|eot_id|><|start_header_id|>assistant<|end_header_id|>

นิทาน "แมว สุนัข และ เต่า"

มีแมว สุนัข และ เต่า 3 ตัว ที่อยู่ในสวนใหญ่ พวกมันคิดว่าตัวเองมีคุณสมบัติที่ดีที่สุดในโลก

แมวคิดว่าตัวเองมีความสามารถในการวิ่งเร็วและกระโดดสูง สุนัขคิดว่าตัวเองมีความสามารถในการวิ่งเร็วและหามนุษย์ให้พบ ส่วนเต่าคิดว่าตัวเองมีความสามารถในการว่ายน้ำและอยู่ในน้ำได้นาน

วันหนึ่ง พวกมันพบกันในสวนและพูดคุยกัน แมวพูดว่า "ฉันคิดว่าตัวเองมีความสามารถที่ดีที่สุดในโลก" สุนัขพูดว่า "ฉันคิดว่าตัวเองมีความสามารถที่ดีที่สุดในโลก" เต่าพูดว่า "ฉันคิดว่าตัวเองมีความสามารถที่ดีทสุดในโลก"

พวกมันกังวลและไม่พอใจกับคำพูดของตัวเอง พวกมันจึงตัดสินใจที่จะไปพบกับพญานาค เพื่อขอคำปรึกษาพวกมันไปพบกับพญานาคและพูดว่า "พญานาค เราคิดว่าตัวเองมีความสามา

In [18]:
# Clean up

del inputs, outputs, model
torch.cuda.empty_cache()

## A couple of quick notes on the next block of code:

I'm using a HuggingFace utility called TextStreamer so that results stream back.
To stream results, we simply replace:  
`outputs = model.generate(inputs, max_new_tokens=80)`  
With:  
`streamer = TextStreamer(tokenizer)`  
`outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)`

Also I've added the argument `add_generation_prompt=True` to my call to create the Chat template. This ensures that Phi generates a response to the question, instead of just predicting how the user prompt continues. Try experimenting with setting this to False to see what happens. You can read about this argument here:

https://huggingface.co/docs/transformers/main/en/chat_templating#what-are-generation-prompts

Thank you to student Piotr B for raising the issue!

In [22]:
# Wrapping everything in a function - and adding Streaming and generation prompts
def generate(model, messages):
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model)
    tokenizer.pad_token = tokenizer.eos_token
    
    # Prepare inputs
    inputs = tokenizer.apply_chat_template(
        messages, 
        return_tensors="pt", 
        add_generation_prompt=True
    ).to("cuda")
    
    # Create a streamer for live output (optional)
    streamer = TextStreamer(tokenizer)
    
    # Load the model
    model = AutoModelForCausalLM.from_pretrained(
        model, 
        device_map="auto", 
        quantization_config=quant_config
    )

    # Generate with temperature=0 and additional parameters
    outputs = model.generate(
        inputs,
        max_new_tokens=5500,
        early_stopping=True,
        temperature=0.1,       # Deterministic behavior
        do_sample=True,       # Typically set False if temperature=0
        top_p=0.95,             # No nucleus sampling constraint
        top_k=50,              # You can adjust this if needed
        repetition_penalty=1.15, # Adjust to penalize repeated phrases
        streamer=streamer ,     # Remove if you don't need live output
        eos_token_id = tokenizer.eos_token_id
    )
    
    # Clean up
    del tokenizer, streamer, model, inputs, outputs
    torch.cuda.empty_cache()


In [20]:
generate(LLAMA, messages)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/transformers/src/transformers/generation/configuration_utils.py:674: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant answer in only thai language<|eot_id|><|start_header_id|>user<|end_header_id|>

เล่านิทานให้ที in thai  เกี่ยวกับ แมว และ สุนัข และ เต่า<|eot_id|><|start_header_id|>assistant<|end_header_id|>

มีนิทานหนึ่งเกี่ยวกับแมว สุนัข และเต่าที่อยู่ในสวนใหญ่ มีแมวที่ชื่อว่า "เมีย" เป็นแมวสีขาวที่ชอบพักผ่อนในทุ่งนา มีสุนัขที่ชื่อว่า "โต้ง" เป็นสุนัขขนาดใหญ่ที่ชอบเล่นกับลูกบอล และมีเตาที่ชื่อว่า "ตวง" เป็นเตาที่ชอบอาบแดดบนหินร้อน

เมียและโต้งเป็นเพื่อนสนิท แต่ก็ไม่เคยพบปะกับตวงเพราะตวงชอบอยู่คนเดียว เมื่อหนึ่งครั้ง โต้งไปหาเมียที่กำลังพักผ่อนในทุ่นา เขาพบว่าตวงกำลังอาบแดดบนหินร้อน โต้งจึงเข้าใกล้ตวงและเรียก "เฮ้ ตวง!" แต่ตวงไม่ได้ยินเลย เพราะกำลังหลับให้สบาย

เมียเห็นตวงกำลังหลับ จึงเข้ามาใกล้และเรียก "เฮ้ ตวง! คุณหลับไปแล้วหรือ?" ตวงได้ยินเสียงของเมียและ睜ตามองรอบๆ แล้วเห็นเมียและสุนัข โต้ง เธอก็ยิ้มและบอก "ฉันหลับไปแล้ว ฉันไม่รู้ว่าคุณสองคนทำอะไร"


In [23]:
generate(Deepseek, messages)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<｜begin▁of▁sentence｜>You are a helpful assistant answer in only thai language<｜User｜>เล่านิทานให้ที in thai  เกี่ยวกับ แมว และ สุนัข และ เต่า<｜Assistant｜><think>
Okay, I need to create a story about animals—cat, dog, and turtle. The user wants it in Thai, so I should make sure the translation is accurate.

First, let me think of a title that's catchy and relevant. Maybe "นิทานสัตว์" which means animal fables or tales.

Next, I'll outline the plot. Perhaps a cat who loves milk but can't reach it, meets a dog who can open jars, and together they find help from a wise turtle.

I should include some moral or lesson, like teamwork and using one's strengths.

Now, translating each part into Thai while keeping the flow natural is important. Let me ensure the sentences make sense and sound good in Thai.

Finally, I'll review the entire story to check for any errors or awkward phrases.
</think>

นิทานสัตว์

มีหนูจุฑาที่ชื่อ "ตัวจัน" เป็นแมวที่รักเมาส์ แต่ไม่สามารถ 达到ถ้วยเมาส์ที่ตั้งบนเสื้อแขวนใ